In [28]:
# Import bibliotek

import pandas as pd 
import numpy as np
import pymysql
import mysql.connector
import datetime as dt
import os
import glob
from datetime import timedelta

print("Script is starting, the current time is", pd.to_datetime("today"))

vicidial = pd.read_excel(r"C:\Users\stefa\Desktop\excele\vicidial\vicidial.xlsx")

now = pd.to_datetime("today")

# Łączę się z magento sql

con = mysql.connector.connect(
  host="prod-replica.ceufrkrz6jt9.eu-central-1.rds.amazonaws.com",
  user="mdziekonski",
  password="ErhoUoD@46%EokESiR@1xq"
)

print(con)
mycursor = con.cursor()

# SQL QUERIES

clients_query =  """select so.customer_email, so.sales_channel, so.entity_id, so.created_at,
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) AS 'customer_max',
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) - so.created_at as max_diff,
                  datediff(MAX(so.created_at) OVER(PARTITION BY so.customer_email), so.created_at) as days_diff,
                  base_subtotal, products, rank() over (partition by so.customer_email order by so.created_at asc) as order_rank
                  from magento.sales_order so
                  where so.state != 'canceled'
                  and so.status != 'canceled'
                  and so.is_valid = 1
                  and so.store_id = 1
                  and so.customer_email is not null
                  and so.customer_email not like '%healthlabs%'
                  and so.customer_email not like '%adequate%'
                  and so.coupon_code NOT IN ('Motywacja15', 'Dietoterapia10', '4US50', '4FR33',
                  'PGFRIENDS35', '4THEM30', 'LORENC', 'LORENC10', 'LORENC25')
UNION ALL
select so.customer_email, so.sales_channel, so.entity_id, so.created_at,
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) AS 'customer_max',
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) - so.created_at as max_diff,
                  datediff(MAX(so.created_at) OVER(PARTITION BY so.customer_email), so.created_at) as days_diff,
                  base_subtotal, products, rank() over (partition by so.customer_email order by so.created_at asc) as order_rank
                  from magento.sales_order so
                  where so.state != 'canceled'
                  and so.status != 'canceled'
                  and so.is_valid = 1
                  and so.store_id = 1
                  and so.customer_email is not null
                  and so.customer_email not like '%healthlabs%'
                  and so.customer_email not like '%adequate%'
                  and so.coupon_code IS NULL"""
clients= pd.read_sql(clients_query, con)

phones_query = ("select phone, email from magento.me_users mu")
phones = pd.read_sql(phones_query, con)

clients = clients[(clients['order_rank'] == 1) & (clients['created_at'] >= '2022-06-21')]
clients['window_days'] = 10
clients['current_date'] = now
clients['threshold'] = pd.to_datetime(clients.current_date) - pd.to_timedelta(pd.np.ceil(clients.window_days), unit="D")
clients = clients[(clients['created_at'] <= clients['threshold'])]

# 60 
# Osoby z poniższymi statusami, które zrobiły ostatnie zamówienie mniej niż 60 dni temu
vicidial_60 = vicidial
filter_list = ['CHUZNR', 'INFOSZ', 'MORE', 'AFTHRS',
'NA','ERI','DROP','DC','B','AFAX','ADC','AB','AA','A']
vicidial_60 = vicidial_60[vicidial_60.status.isin(filter_list)]
vicidial_60['current_date'] = now 
vicidial_60['current_date'] = pd.to_datetime(vicidial_60['current_date'])
vicidial_60['window_days'] = 60
vicidial_60['date_60'] = pd.to_datetime(vicidial_60.current_date) - pd.to_timedelta(pd.np.ceil(vicidial_60.window_days), unit="D")
vicidial_60 = vicidial_60[(vicidial_60['call_date'] >= vicidial_60['date_60'])]
vicidial_exclusion_60 = vicidial_60.drop_duplicates('phone_number_dialed')
vicidial_exclusion_60 = (vicidial_exclusion_60[['phone_number_dialed']])
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_60['exx'] = 1
vicidial_exclusion_60

# 150
# Osoby z poniższymi statusami, które zrobiły ostatnie zamówienie mniej niż 150 dni temu

vicidial_150 = vicidial
filter_list = ['REZBEF','REZCEN','REZINP','REZMAP','REZNST','REZYGN', 'REZWTK', 'REZCIA', 'REZKON', 'REZNIE', 'REZKNT', 'REZPWS', 'REZTZP']
vicidial_150 = vicidial_150[vicidial_150.status.isin(filter_list)]
vicidial_150['window_days'] = 150
vicidial_150['current_date'] = now 
vicidial_150['current_date'] = pd.to_datetime(vicidial_150['current_date'])
vicidial_150['date_150'] = pd.to_datetime(vicidial_150.current_date) - pd.to_timedelta(pd.np.ceil(vicidial_150.window_days), unit="D")
vicidial_150 = vicidial_150[(vicidial_150['call_date'] >= vicidial_150['date_150'])]
vicidial_exclusion_150 = vicidial_150.drop_duplicates('phone_number_dialed')
vicidial_exclusion_150 = (vicidial_150[['phone_number_dialed']])
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_150['exx'] = 1
vicidial_exclusion_150

# Wykluczenia 
# Osoby, które nie wyrażają zgody żeby do nich dzwonić

nichko = vicidial
filter_list = ['DUBEL','NICHKO','ODMROZ','OSNIDO','POM',
#'PROPLA',
'ZASTAN'
#,'ODD'
]
nichko = nichko[nichko.status.isin(filter_list)]
nichko['current_date'] = now 
nichko['current_date'] = pd.to_datetime(nichko['current_date'])
nichko['window_days'] = 20000
nichko['date_20000'] = pd.to_datetime(nichko.current_date) - pd.to_timedelta(pd.np.ceil(nichko.window_days), unit="D")
nichko = nichko[(nichko['call_date'] >= nichko['date_20000'])]
vicidial_exclusion_nich = nichko.drop_duplicates('phone_number_dialed')
vicidial_exclusion_nich = (nichko[['phone_number_dialed']])
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_nich['exx'] = 1
vicidial_exclusion_nich

# Połączenie Wykluczeń
# Łączymy 4 powyższe grupy, są to osoby do których nie dzwonimy

vicidial_exclusion = pd.concat([vicidial_exclusion_nich, vicidial_exclusion_60, vicidial_exclusion_150])
vicidial_exclusion = vicidial_exclusion.drop_duplicates()
vicidial_exclusion


# Dataset Creation

df_main = clients
df_main = df_main[['customer_email', 'created_at', 'current_date']]

phones2 = phones
phones2['phone']=phones2['phone'].astype(str)
phones2['phone'] = phones2['phone'].str.replace(r'\+48', '', regex=True)
phones2['phone'] = phones2['phone'].str.replace(r' ', '', regex=True)
phones2.phone = pd.to_numeric(phones2.phone, errors='coerce').fillna(0).astype(np.int64)
phones2 = phones2[phones2['phone'].between(400000000, 960000000)]
phones2

phones2 = phones2.set_index('email')
df_main2 = df_main.set_index('customer_email')
df_main2 = df_main.merge(phones2, left_on='customer_email', right_on='email', how='left')
df_main2 = df_main2[df_main2['phone'].notna()]
df_main2 = df_main2.set_index('phone')

vicidial_exclusion = vicidial_exclusion.set_index('phone_number_dialed')
df_main2 = df_main2.merge(vicidial_exclusion, left_index=True, right_index=True, how='left')
df_main2 = df_main2.reset_index()

df_final = df_main2[(df_main2['exx'].isnull())]
df_final = df_final.sort_values(by='created_at', ascending=True)
df_final = df_final.rename(columns={'index': 'phone'})
df_final = df_final[['phone', 'created_at']]

# path = r'C:\Users\stefa\Desktop\csvki\opieka'  
# all_files = glob.glob(os.path.join(path, "*.csv"))     
# df_from_each_file = (pd.read_csv(f) for f in all_files)
# concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
# concatenated_df

# Dołączenie  do poprzedniego, finałowego datasetu

# df_final2 = df_final.merge(concatenated_df, left_on='phone', right_on='phone2', how='left')

# Wyrzucenie duplikatów, czyli numerów, które były w poprzednich datasetach

# df_final2 = df_final2[df_final2['phone2'].isnull()]
# df_final2 = df_final2[['phone', 'Score_czas', 'Score_historia', 'Score_suma']]

# Ostateczna baza wszystkich klientów, posortowana wg scoru
df_final


Script is starting, the current time is 2022-07-11 16:57:24.126916


C:\Users\stefa\AppData\Local\Temp/ipykernel_52868/470581698.py:69: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  clients['threshold'] = pd.to_datetime(clients.current_date) - pd.to_timedelta(pd.np.ceil(clients.window_days), unit="D")
C:\Users\stefa\AppData\Local\Temp/ipykernel_52868/470581698.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vicidial_60['current_date'] = now
C:\Users\stefa\AppData\Local\Temp/ipykernel_52868/470581698.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,phone,created_at
400,517723943.0,2022-06-21 02:44:03
785,609407612.0,2022-06-21 03:21:19
1220,780091605.0,2022-06-21 04:57:16
1367,796845073.0,2022-06-21 06:01:59
771,608625018.0,2022-06-21 06:23:14
...,...,...
16,500450886.0,2022-07-01 15:28:43
1321,792607582.0,2022-07-01 15:36:34
798,660206826.0,2022-07-01 15:41:23
368,515422613.0,2022-07-01 16:29:15


In [ ]:
df_final2.to_csv('Baza_opieka_20220706.csv')
